# Soda Reconcilliation Check
Reconcile datasets after copying from PostgreSQL to Synapse SQL.

## Configure connections to the data source and Soda Cloud
For sensitive credential values, this example gets the values from a linked Azure Key Vault.

In [2]:
from notebookutils import mssparkutils

config_str = f"""
data_source postgres_data:
  type: postgres
  host: soda-demo.xxx.eu-west-1.rds.amazonaws.com
  port: 5432
  username: my_user
  password: {mssparkutils.credentials.getSecret('soda-vault' , 'postgres-pw')}
  database: postgres
  schema: soda_demo_data_testing
data_source azure_sql_data:
  type: sqlserver
  driver: ODBC Driver 18 for SQL Server
  host: soda.sql.azuresynapse.net
  port: xxxx
  username: my_sql_user
  password: {mssparkutils.credentials.getSecret('soda-vault' , 'sql-pw')}
  database: soda_sqlserver
  schema: soda_demo_data_testing
soda_cloud:
  host: cloud.us.soda.io
  api_key_id: {mssparkutils.credentials.getSecret('soda-vault' , 'soda-api-key-id')}
  api_key_secret: {mssparkutils.credentials.getSecret('soda-vault' , 'soda-api-key-secret')}
"""

StatementMeta(tasoda, 19, 3, Finished, Available, Finished)

## Define the reconcilliation checks using Soda Checks Language (SodaCL)
This section defines the Soda checks for the datasets. Refer to Soda docs for details: https://go.soda.io/recon

In [26]:
check_str = """reconciliation retail_customers:
  label: 'Reconcile Postgres source and Azure SQL target'
  datasets:
    source:
      dataset: retail_customers
      datasource: postgres_data
    target:
      dataset: retail_customers
      datasource: azure_sql_data

  checks:
    - row_count diff = 0
    - duplicate_count(customer_id):
        fail: when diff > 0
    - missing_count(customer_id):
        fail: when diff > 0
    - missing_count(country_code):
        fail: when diff > 0
"""

StatementMeta(tasoda, 19, 27, Finished, Available, Finished)

## Run the Soda scan

If `scan.assert_no_checks_fail()` returns an `AssertionError` due to failed checks, then the Azure Data Factory pipeline in which this notebook resides halts.

In [27]:
from soda.scan import Scan
scan = Scan()
scan.set_data_source_name('azure_sql_data')
scan.add_configuration_yaml_str(config_str)
scan.set_scan_definition_name('reconciliation')
scan.set_verbose(True)
scan.add_sodacl_yaml_str(check_str)
scan.execute()
scan.assert_no_checks_fail()

StatementMeta(tasoda, 19, 28, Finished, Available, Finished)